In [1]:
import os
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
from doublegis_api.api import Api2Gis

api = Api2Gis()
api.load()

In [ ]:
main, sub = get_filter_cats_ids(api.rubrics, api.sub_rubrics)
api.filials = filter_by_categories(api.filials, api.organizations, main, sub)

In [5]:
import numpy as np
import pandas as pd
import dateutil.parser
from datetime import datetime
from itertools import compress


from ipywidgets import IntProgress
from IPython.display import display
from doublegis_api.metro_coordinates import get_nearest_stations
from filtering.categories import get_data_by_filial, get_filter_cats_ids, filter_by_categories


def excel_processing(api):
    data = []
    progress = IntProgress(min=0, max=len(api.filials), value=0)
    display(progress)
    for i, filial in enumerate(api.filials):
        try:
            filial, org, main_rubric, sub_rubrics = get_data_by_filial(api, filial)
        except:
            continue
        metro, distance = get_nearest_stations(filial, api.metro_stations)[0]
        if distance > 1000:
            continue
        
        living_period = None
        if filial.created_at_json['2gis_appear_at'] != '':
            created_at = dateutil.parser.parse(filial.created_at_json['2gis_appear_at']).replace(tzinfo=None)
            if filial.closed_at_json['2gis_removed_at'] != '':
                removed_at = dateutil.parser.parse(filial.closed_at_json['2gis_removed_at']).replace(tzinfo=None)
                living_period = (removed_at - created_at).days // 31
            else:
                living_period = (datetime.today() - created_at).days // 31
        
        data.append([main_rubric.name, org.name_primary, filial.address_synonyms[0], filial.latitude, 
                     filial.longitude, metro.name, metro.latitude, metro.longitude, distance / 1000.,
                     filial.created_at_json['2gis_appear_at'], filial.closed_at_json['2gis_removed_at'],
                     living_period, 1 if filial.closed_at_json['2gis_removed_at'] == '' else 0])
        progress.value = i
        
    return pd.DataFrame(data=data,
                        columns=['category', 'company', 'address', 'latitude', 'longtitude', 'nearest_name',
                                 'nearest_metro_lat', 'nearest_metro_long', 'nearest_distance',
                                 'created_date', 'removed_date', 'living_period', 'current_status'])

excel_processed = excel_processing(api)
excel_processed.rename(columns={'category': 'Отрасль',
                                'company': 'Компания',
                                'address': 'Адрес',
                                'latitude': 'Широта_к',
                                'longtitude': 'Долгота_к',
                                'nearest_name': 'Ближайшее метро',
                                'nearest_metro_lat': 'Широта_м',
                                'nearest_metro_long': 'Долгота_м',
                                'nearest_distance': 'Расстояние до метро',
                                'created_date': 'Дата открытия',
                                'removed_date': 'Дата закрытия',
                                'living_period': 'Срок жизни, мес.',
                                'current_status': 'Статус'}, inplace=True)
excel_processed

,Отрасль,Компания,Адрес,Широта_к,Долгота_к,Ближайшее метро,Широта_м,Долгота_м,Расстояние до метро,Дата открытия,Дата закрытия,"Срок жизни, мес.",Статус
0,Досуг / Развлечения / Общественное питание,Санкт-Петербург,"Набережная канала Грибоедова, 5",59.939860,30.327718,Невский проспект,59.935601,30.327134,0.474,,,NaN,1
1,Медицина / Здоровье / Красота,Jean Vallon,"Озерной переулок, 2-4",59.938179,30.361802,Чернышевская,59.944558,30.359754,0.718,,,NaN,1
2,Медицина / Здоровье / Красота,Jean Vallon,"Малая Конюшенная, 4",59.937883,30.326631,Невский проспект,59.935601,30.327134,0.255,,,NaN,1
3,Реклама / Полиграфия / СМИ,Where St.Petersburg,"Торжковская, 1 к2",59.986717,30.306038,Чёрная речка,59.985574,30.300792,0.318,,,NaN,1
4,Досуг / Развлечения / Общественное питание,Франсманни,"Батайский переулок, 3а",59.913504,30.320728,Технологический институт-2,59.916228,30.318407,0.329,,,NaN,1
5,Культура / Искусство / Религия,На Коломенской,"Коломенская, 43",59.920837,30.349349,Лиговский проспект,59.920747,30.355245,0.328,,,NaN,1
6,Электроника / Электротехника,"Буревестник, АО","Стахановцев, 1",59.930120,30.405122,Новочеркасская,59.929330,30.412918,0.443,,,NaN,1
7,Спорт / Отдых / Туризм,Renaissance St.Petersburg baltic hotel,"Почтамтская, 4",59.932843,30.304568,Адмиралтейская,59.935941,30.314872,0.669,,,NaN,1
8,Электроника / Электротехника,"Ритверц, ЗАО","Курчатова, 10",60.008808,30.361974,Политехническая,60.008926,30.370952,0.499,,,NaN,1
9,Электроника / Электротехника,Синтез НДТ,"Гжатская, 21 к1",60.005450,30.385038,Политехническая,60.008926,30.370952,0.873,,,NaN,1


In [6]:
excel_processed.to_excel('test.xlsx')